In [ ]:
%%capture
!pip install -q unsloth # install unsloth. automatically resolve dependencies
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## 이 워크스루를 통해 모든 관련 패키지 가져온다.

In [ ]:
!pip install -q datasets trl

In [ ]:
# 파인튜닝 모듈
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
# 허깅페이스 모듈
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## API 키 생성 및 허깅 페이스와 wandb에 로그인하기

In [ ]:
import os
hugging_face_token = "허깅페이스 토큰 입력"  # 너의 Hugging Face 토큰 입력
wnb_token = "완드비 토큰 입력"

In [ ]:
# 허깅페이스 & WnB 토큰 초기화하기
login(hugging_face_token)

# # WnB 로그인하기
wandb.login(key=wnb_token)
run = wandb.init(
project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset_YouTube Walkthrough',
job_type="training",
anonymous="allow"
)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 20221335 (20221335-sungshin-women-s-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 딥시크 R1 토크나이저 로드

In [ ]:
#파라미터 설정
max_seq_length = 2048 # 모델이 처리할 수 있는 최대 시퀀스 길이
dtype = None # 기본 데이터 유형
load_in_4bit = True # 4bit 양자화 활성화 - 메모리 절약 최적화

# DeepSeek R1 모델 및 토크나이저 로드 - unsloth로부터 fastlanguagemodel 가져오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
    token=os.environ.get("hugging_face_token"),
)

==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 파인튜닝 전 의료 데이터로 DeepSeek R1 테스트하기



In [ ]:
# 프롬프트 정의
prompt_style = """아래는 특정 작업에 대한 지침과 함께, 추가적인 맥락을 제공하는 입력이 주어져 있습니다.
요청에 적절히 응답하는 답변을 작성하세요.
답변을 작성하기 전에 질문을 신중히 분석하고, 단계적인 사고 과정을 통해 논리적이고 정확한 결론에 도달하세요.

### 지침:
당신은 임상 추론, 진단, 치료 계획에 대해 고도의 전문 지식을 갖춘 의료 전문가입니다.
다음 의학 질문에 대해 **한국어로만** 답변해 주세요.

### 질문:
{}

### 응답:
<think>{}"""


### 모델에서 추론 실행하기


1. 테스트 질문 정의
2. 모델이 논리적 추론 과정을 따르도록 구조화된 프롬프트 사용하여 문제 형식화
3. 입력 토큰화하여 추론 속도 높임
4. 모델 사용하여 응답 생성
5. 출력 토큰을 다시 텍스트로 디코딩

In [ ]:
# 테스트 의료 질문 예시
question = """61세 여성이 기침이나 재채기 같은 활동 중에 발생하는 오랜 요실금 병력이 있으나,
밤에는 누출이 없습니다. 부인과 검사와 Q-tip 테스트를 시행하였습니다.
이러한 소견에 근거할 때, 방광내압검사는 잔뇨량과 배뇨근 수축에 대해 어떤 결과를 보여줄 가능성이 높을까요?"""

# 추론 모드 활성화
FastLanguageModel.for_inference(model)

# 구조화된 프롬프트를 사용하여 질문의 형식을 지정하고 토큰화
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 모델 사용하여 응답 생성
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# 디코딩
response = tokenizer.batch_decode(outputs)

# 관련 응답 부분(“### 응답:” 뒤)만 추출
print(response[0].split("### 응답:")[1])


<think>
好的，我现在要帮助这个61岁的女性医生制定治疗计划。首先，了解她的症状情况：她有长时间的尿潴留，但晚上没有尿意或排尿问题。这可能提示她可能有狭窄的尿道或其他结构问题，但排尿中枢正常。已经做了夫妻检查和Q-tip测试，这些检查可能已经排除了狭窄的尿道或尿道功能异常。接下来，考虑进行膀胱压力测试（CPT），这个测试主要用于评估膀胱内压力，以判断是否有膀胱功能障碍。膀胱内压力过高可能导致排尿困难，但如果膀胱压力正常，可能需要进一步检查尿道结构或其他系统。因此，CPT的结果可能显示膀胱内压力是否正常，从而指导后续治疗步骤。因此，最有可能显示的结果是膀胱内压力正常，这提示需要考虑其他原因，如尿道狭窄或其他结构问题。
</think>

61세 여성은 요실금(尿潴留)으로 인해 부인과 검사와 Q-tip 테스트를 이미 시행한 상태입니다. 이러한 소견에 근거할 때, 방광내압검사(膀胱内압力 검사, CPT)에서 잔뇨량과 배뇨근 수축에 대한 결과를 보여줄 가능성이 높습니다.

### 이유:
- **尿潴留(요실금)**: 그녀는长时间에 걸쳐 요실금(尿潴留)병력을 가지고 있습니다. 이는 배아와膀胱에 압박을 주는 증상이 될 수 있습니다.
- **Q-tip 테스트**: Q-tip 테스트는 배아와尿道에 대한 구조적 문제(예:尿道狭窄)를 평가하는 데 사용됩니다. 이 테스트는 그녀가 배아나尿道에 구조적인 문제를 가지고 있는지 아닌지를 확인합니다.
- **膀胱内압力 검사(CPT)**: CPT는 배아와膀胱의 내압을 측정하는 데 사용됩니다. 이 검사에서 잔뇨량과 배뇨근 수축에 대한 결과를 보여줄 수 있습니다. CPT는 배아와膀胱의 내압이正常인지 아니지 않음을 확인할 수 있습니다.
- **잔뇨량과 배뇨근 수축**: 이两个는 배아와膀胱의功能을 유지하는 데 중요한 구조입니다. CPT에서 이 두部分의 수축과 잔뇨량이正常인지 아닌지 확인할 수 있습니다.

### 결과:
CPT에서 잔뇨량과 배뇨근 수축이正常이면, 이는 배아와膀胱의 기능이 정상적인 상태를 지지한다는 의미입니다. 만약 이 두部分이异常이라면, 이는 배아와膀胱의 기능障碍(예:배아狭窄)을示唤할 수 있습니

# 모델 시작

In [ ]:
# </think> 태그를 추가하도록 교육 프롬프트 업데이트
train_prompt_style = """아래는 특정 작업에 대한 지침과 함께, 추가적인 맥락을 제공하는 입력이 주어져 있습니다.
요청에 적절히 응답하는 답변을 작성하세요.
답변을 작성하기 전에 질문을 신중히 분석하고, 단계적인 사고 과정을 통해 논리적이고 정확한 결론에 도달하세요.

### 지침:
당신은 임상 추론, 진단, 치료 계획에 대해 고도의 전문 지식을 갖춘 의료 전문가입니다.
다음 의학 질문에 대해 **한국어로만** 답변해 주세요.

### 질문:
{}

### 응답:
<think>
{}
</think>
{}"""


## 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import Dataset
import json
import re

# 1. 병합된 JSON 경로 지정
json_path = "/content/drive/MyDrive/의료데이터셋/combined_translated_0000_3999.json"

# 2. JSON 로드
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 3. 한자(중국어)가 포함되었는지 확인하는 함수
def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', text))

# 4. 유효 항목만 골라내고, 중국어 제거
cleaned_data = []
skipped_chinese = 0
skipped_incomplete = 0

for item in data:
    if all(k in item for k in ["Question_ko", "Complex_CoT_ko", "Response_ko"]):
        q, c, r = item["Question_ko"], item["Complex_CoT_ko"], item["Response_ko"]
    elif all(k in item for k in ["Question", "Complex_CoT", "Response"]):
        q, c, r = item["Question"], item["Complex_CoT"], item["Response"]
    else:
        skipped_incomplete += 1
        continue

    # 한자 포함 여부 확인
    if contains_chinese(q) or contains_chinese(c) or contains_chinese(r):
        skipped_chinese += 1
        continue

    cleaned_data.append({
        "Question": q,
        "Complex_CoT": c,
        "Response": r
    })

print(f"✅ 최종 유효한 항목 수: {len(cleaned_data)}")
print(f"🚫 제거된 중국어 포함 항목 수: {skipped_chinese}")
print(f"🚫 키 누락 등으로 제외된 항목 수: {skipped_incomplete}")

# 5. 🤗 Hugging Face Dataset으로 변환
dataset = Dataset.from_list(cleaned_data)


✅ 최종 유효한 항목 수: 3987
🚫 제거된 중국어 포함 항목 수: 15
🚫 키 누락 등으로 제외된 항목 수: 0


In [ ]:
dataset[1]

{'Question': '33세 여성이 드라이버로 가슴을 찔린 지 15분 후 응급실로 후송되었습니다.  맥박 110/분, 호흡 22/분, 혈압 90/65 mmHg의 활력징후와 좌측 겨드랑이 중앙선 8번째 갈비뼈 상연에 깊이 5cm의 자상이 있는 것을 고려할 때, 그녀의 가슴에서 가장 손상되었을 가능성이 높은 해부학적 구조는 무엇입니까?',
 'Complex_CoT': '자, 여기서 무슨 일인지 알아봅시다. 한 여성이 드라이버에 찔린 상처를 입고 왔습니다.  흉부, 왼쪽 8번째 갈비뼈 상연, 겨드랑이 중간선(midaxillary line) 부근입니다.  일단 생각해보면, 폐가 있는 곳과 매우 가깝죠?\n\n먼저 위치부터 이야기해 봅시다.  왼쪽 몸통에 위치하고 있으며, 8번째 갈비뼈 위쪽, 그 부위에는 좌측 폐 하엽과 횡경막 등 중요한 장기들이 많이 있습니다. 드라이버가 얼마나 깊이 들어갔는지 고려하면 횡경막도 포함될 가능성이 있습니다.\n\n상처 깊이는 5cm입니다.  꽤 깊네요. 폐 조직이나 횡경막까지 도달했을 가능성이 분명히 있습니다. 겨드랑이 중간선(midaxillary) 부위라는 점을 고려하면, 좌측 폐 하엽이 있는 영역입니다. 횡경막과 일부 겹칠 가능성도 있지만, 폐가 더 유력해 보입니다.\n\n이제 활력징후가 문제입니다. 심박수가 높고 혈압이 낮습니다. 심각하네요.  제 생각에는 이런 활력징후는 pneumothorax 또는 hemothorax를 의미할 수 있습니다. 폐가 손상되면 두 가지 모두 발생할 수 있으며, 환자는 분명 고통스러울 것이므로 혈압을 떨어뜨리고 심박수를 치솟게 할 수 있습니다.\n\n그러니까, 종합해 보면 가장 의심되는 것은 좌측 폐 하엽입니다. 상처의 깊이와 환자의 상태가 이를 뒷받침합니다.  그리고 이것은 pneumothorax 또는 흉부에 출혈—호흡과 순환에 심각한 문제를 일으킬 수 있는 것들—과 일치합니다.\n\n좋아요, 이를 고려해 볼 때, 가장 손상될 가능성이 높은 것은 폐인 것 같습니다. 상처 위치, 증상, 그리고 전체

## 미세조정 데이터셋 구성

In [ ]:
# 훈련 중 텍스트 생성 중지 시점을 모델에 정의하는 EOS_TOKEN을 정의
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [ ]:
# 프롬프트 기능 정의
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]

    texts = []

    # 데이터 집합을 반복하여 각 질문, 추론 단계 및 응답의 서식을 지정
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/3987 [00:00<?, ? examples/s]

"아래는 특정 작업에 대한 지침과 함께, 추가적인 맥락을 제공하는 입력이 주어져 있습니다.\n요청에 적절히 응답하는 답변을 작성하세요.\n답변을 작성하기 전에 질문을 신중히 분석하고, 단계적인 사고 과정을 통해 논리적이고 정확한 결론에 도달하세요.\n\n### 지침:\n당신은 임상 추론, 진단, 치료 계획에 대해 고도의 전문 지식을 갖춘 의료 전문가입니다.\n다음 의학 질문에 대해 **한국어로만** 답변해 주세요.\n\n### 질문:\n왼쪽 팔과 다리의 갑작스러운 약화(sudden weakness), 최근 장거리 여행(recent long-distance travel), 그리고 부어오르고 압통이 있는 오른쪽 아랫다리(swollen and tender right lower leg) 증상을 고려할 때, 이러한 소견을 설명할 수 있는 가장 가능성 높은 특정 심장 이상(cardiac abnormality)은 추가 검사(further evaluation)에서 무엇일까요?\n\n### 응답:\n<think>\nOkay, let's see what's going on here.  환자의 왼쪽 팔과 다리에 갑작스러운 weakness가 나타났습니다 - 이건 신경계 관련 문제, 어쩌면 stroke?을 시사합니다.\n\n하지만, 그게 다가 아닙니다. 오른쪽 아래 다리가 부어오르고 tender합니다. 이건 특히 장시간 비행이나 오랫동안 앉아 있은 후에, deep vein thrombosis를 강하게 의심하게 만드는 증상입니다.\n\n그러니 이제 생각해 봐야 합니다. 다리의 clot이 어떻게 weakness나 stroke 증상 같은 문제를 일으킬 수 있을까요?\n\n아, 맞다! paradoxical embolism이라는 것이 있습니다. 심장에 어떤 종류의 short circuit, 예를 들어 있어서는 안 될 구멍이 있을 경우 발생할 수 있습니다.\n\n정리해 봅시다: 다리의 혈전이 어떻게든 심장의 왼쪽으로 이동하면, 뇌로 이동하여 혈류를 차단하고 그 갑작스러운 weakness를 일으

### LoRA를 사용하여 모델 설정

모든 가중치를 수정하는 대신, **LoRA는 특정 레이어에 훈련 가능한 소형 어댑터**를 추가한다.  
- 이러한 어댑터는 원래 모델을 변경하지 않고 작업별 지식을 **캡쳐**한다.  
- 이를 통해 학습 가능한 파라미터의 수를 90% 이상** 줄여 미세 조정을 **더 빠르고 메모리 효율적으로** 할 수 있다.

In [ ]:
# Apply LoRA
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # 학습 가능한 어댑터의 크기를 결정
    target_modules=[  #  LoRA 어댑터가 적용될 트랜스포머 레이어 목록
        "q_proj",   # Query projection
        "k_proj",   # Key projection
        "v_proj",   # Value projection
        "o_proj",   # Output projection
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # LoRA 업데이트에 대한 스케일링 계수(값이 클수록 LoRA 레이어의 영향력이 커짐)
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,  #등급 안정화 LoRA 사용 여부(여기서는 비활성화되어 고정 등급 LoRA가 사용됨을 의미
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) 비활성화
)

Unsloth 2025.5.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import types

In [ ]:
# 문제가 되는 부분을 피하기 위한 새로운 `for_training` 메서드 정의
def for_training_patched(self, use_gradient_checkpointing=True):
    """Sets the model to training mode and optionally enables gradient checkpointing."""
    self.train()  # enable training mode

# 새로운 `for_training` 메서드를 `model_lora` 객체에 동적으로 추가
model_lora.for_training = types.MethodType(for_training_patched, model_lora)

SFTTrainer 초기화

In [ ]:
trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=dataset_finetune,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,

    # 학습 파라미터 정의
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,  # 8스텝 동안 gradient를 누적 후 가중치 업데이트
        num_train_epochs=3,
        warmup_steps=5,  # 처음 5스텝 동안 학습률을 점진적으로 증가
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",  # 메모리를 절약하는 8비트 AdamW 옵티마이저 사용
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to='none',
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/3987 [00:00<?, ? examples/s]

## 모델 학습


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,987 | Num Epochs = 3 | Total steps = 1,497
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.454400
20,1.852400
30,1.670300
40,1.624800
50,1.556900
60,1.510300
70,1.495200
80,1.506200
90,1.507800
100,1.443200


In [ ]:
trainer_stats

TrainOutput(global_step=1497, training_loss=1.2037637107915375, metrics={'train_runtime': 19440.788, 'train_samples_per_second': 0.615, 'train_steps_per_second': 0.077, 'total_flos': 4.897552329502802e+17, 'train_loss': 1.2037637107915375})

In [ ]:
model_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
model_lora.push_to_hub("nanyaas/deepseek-r1-medicalQA-Qwen_first")
tokenizer.push_to_hub("nanyaas/deepseek-r1-medicalQA-Qwen_first")

Saved model to https://huggingface.co/nanyaas/deepseek-r1-medicalQA-Qwen_first


In [ ]:
new_model_name = "nanyaas/deepseek-r1-medicalQA-Qwen_first"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model);

==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## 파인튜닝 완료된 모델로 추론 수행하기

In [ ]:
%%time
question = """61세 여성이 기침이나 재채기 같은 활동 중에 발생하는 오랜 요실금 병력이 있으나,
밤에는 누출이 없습니다. 부인과 검사와 Q-tip 테스트를 시행하였습니다.
이러한 소견에 근거할 때, 방광내압검사는 잔뇨량과 배뇨근 수축에 대해 어떤 결과를 보여줄 가능성이 높을까요?"""

FastLanguageModel.for_inference(model_lora)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("<think>")[-1].strip())

자, 61세 여성 환자가 있습니다.  오랫동안 요실금을 겪고 있는데, 기침이나 재채기와 같은 활동을 할 때 악화됩니다. 흥미로운 점은 밤에는 소변을 누르지 않아도 소변을 잘 보관한다는 것입니다.  이는 중요한 단서입니다.

먼저, 이러한 요실금의 원인이 무엇일지 생각해 봅시다. 요실금은 주로 요도 또는 방광의 기능이나 구조적 문제로 인해 발생할 수 있습니다.  잠깐만요,  요실금이 기침이나 재채기와 같은 특정 활동과 관련이 있다는 점을 고려하면,  방광이 비정상적으로 아래쪽으로 처져 있을 가능성이 있습니다. 이러한 방광 아래쪽 이동은 요도와 방광의 위치에 영향을 미치는 요골근의 기능 이상을 시사합니다.

이제, 방광 내압 검사(cystometric study)가 있습니다.  방광 내압을 측정하고 방광을 비우는 동안 방광을 유지하는 능력을 확인하는 검사입니다.  방광이 아래로 처져 있으면, 방광 내압이 상승할 수 있습니다.  이는 방광이 아래로 처지면 자체적 저항 때문에 더 높은 압력이 축적될 수 있기 때문입니다.

하지만 잠깐,  방광 내압이 높더라도, 이 환자의 경우 밤에는 누출이 없으므로,  방광 내압이 높더라도 배뇨 시 방광이 수축하는 능력이 문제가 아닌 것 같습니다.  방광 내압이 높더라도 배뇨근이 수축하는 데는 문제가 없다면,  배뇨 시 방광이 제대로 수축하는 것이 중요합니다.

따라서,  방광 내압이 높더라도 배뇨근이 수축하는 데는 문제가 없다면,  배뇨근 기능은 정상일 가능성이 높습니다.  방광 내압이 높더라도 배뇨근의 수축 기능이 정상이라면,  방광 내압과 배뇨근 기능 모두 정상일 수 있습니다.

아,  Q-tip test도 있습니다.  이는 방광 아래쪽이 처지면 방광의 움직임에 도움이 되는 요골근의 기능을 확인하는 데 사용됩니다.  Q-tip test 결과가 양성이라면,  요골근의 기능이 정상적으로 작용하여 방광이 아래쪽으로 처지더라도 배뇨 시 정상적으로 이동한다는 것을 의미합니다.

따라서,  Q-tip test 결과가 양성이고 배뇨근 기능이

In [ ]:
%%time
question = """59세 남성이 발열, 오한, 야간 발한, 전신 피로감을 호소하며 내원하였고,
대동맥판에서 12mm의 vegetation이 발견되었습니다. 혈액 배양 검사 결과는
gram-positive, catalase-negative, gamma-hemolytic cocci가 사슬 모양으로 관찰되었으며,
6.5% NaCl 배지에서는 자라지 않았습니다.
이 환자의 상태에 가장 관련 있는 선행 요인은 무엇일까요?"""

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

print(response[0].split("<think>")[-1].strip())

알겠습니다, 무슨 일인지 살펴봅시다. 59세 남성이 발열, 오한, 야간 발한과 같은 전형적인 증상들을 보이고 있습니다.  꽤 심각해 보이고, 전신 피로감을 호소하고 있는데, 이는 단순한 감기와는 다릅니다.  더 심각한 질병일 가능성이 높습니다.

이제, 심장 판막, 특히 대동맥판에서 12mm 크기의 덩어리가 발견되었습니다.  이는 매우 심각한 징후입니다. 덩어리는 어떤 종류의 감염, 아마도 대동맥 판막의 감염일 가능성이 높습니다.

그리고, 혈액 배양 검사를 했는데, 그 결과는 gram-positive, catalase-negative, gamma-hemolytic cocci로 나타났습니다.  이는 특정 세균을 가리킵니다.  gram-positive, catalase-negative, gamma-hemolytic인 세균은 *Streptococcus viridans*와 *Enterococcus faecalis*와 같은 특정 세균을 생각하게 합니다.

하지만, 6.5% NaCl 배지에서 세균이 자라는 것을 보지 못했습니다.  이는 중요한 단서입니다. 일반적으로 *Enterococcus faecalis*는 6.5% NaCl 배지에서 자랍니다. 따라서, 이 세균은 아닙니다.

이제, *Streptococcus viridans*는 특히 고령자, 특히 심장 판막 문제가 있는 사람들에게서, 감염성 심내막염이나 대동맥 판막의 덩어리와 같은 합병증과 관련이 있습니다.

따라서, 이 모든 것을 종합해 볼 때, 환자의 상태는 *Streptococcus viridans*에 의한 감염성 심내막염과 매우 일치합니다.  이 세균은 심장 판막의 감염을 일으키는 것으로 알려져 있으며,  우리가 보고 있는 것과 같은 덩어리를 유발할 수 있습니다.

또한, 환자의 나이와 증상은 이 진단을 더욱 뒷받침합니다.  나이, 증상, 배양 결과를 종합해 볼 때, 감염성 심내막염이 가장 가능성이 높은 원인입니다.

따라서, 이 모든 것을 고려할 때, 이 환자의 상태는 *Streptococcus virida

# **검증하기**

In [ ]:
from datasets import Dataset
import json
import re

# 1. 두 JSON 파일 경로
json_paths = [
    "/content/drive/MyDrive/의료데이터셋/translated_4001_4100_ko_only.json",
    "/content/drive/MyDrive/의료데이터셋/translated_4101_4200_ko_only.json",
]

# 2. 한자(중국어)가 포함되었는지 확인하는 함수
def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', text))

def remove_cot(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

# 3. 병합 및 클렌징
cleaned_data = []
skipped_chinese = 0
skipped_incomplete = 0

for path in json_paths:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for item in data:
        if all(k in item for k in ["Question_ko", "Complex_CoT_ko", "Response_ko"]):
            q, c, r = item["Question_ko"], item["Complex_CoT_ko"], item["Response_ko"]
        elif all(k in item for k in ["Question", "Complex_CoT", "Response"]):
            q, c, r = item["Question"], item["Complex_CoT"], item["Response"]
        else:
            skipped_incomplete += 1
            continue

        if contains_chinese(q) or contains_chinese(c) or contains_chinese(r):
            skipped_chinese += 1
            continue

        r_clean = remove_cot(r)

        cleaned_data.append({
            "Question": q.strip(),
            "Response": r_clean.strip(),
        })

# 4. 통계 출력
print(f"✅ 최종 유효한 항목 수: {len(cleaned_data)}")
print(f"🚫 제거된 중국어 포함 항목 수: {skipped_chinese}")
print(f"🚫 키 누락 등으로 제외된 항목 수: {skipped_incomplete}")

# 5. 🤗 Hugging Face Dataset 변환
dataset = Dataset.from_list(cleaned_data)

# 필요 시 확인
print(dataset[0])


✅ 최종 유효한 항목 수: 198
🚫 제거된 중국어 포함 항목 수: 2
🚫 키 누락 등으로 제외된 항목 수: 0
{'Question': '2살짜리 아이가 발열과 구토, 그리고 경부 강직 증상을 보이며 응급실에 왔습니다. 뇌척수액 검사 결과 백혈구 수는 2000/µL 이고 단백질 수치는 100 mg/dL 입니다. 그람염색 결과 그람음성 간균이 나타났습니다. 세균 배양은 초콜릿 한천 배지에서는 자라지만 혈액 한천 배지에서는 자라지 않았습니다. 이 아이의 증상의 가장 가능성이 높은 원인균은 무엇입니까?', 'Response': '이 아이의 증상을 유발한 가장 가능성 높은 원인균은 *Haemophilus influenzae*입니다. 임상 증상, 뇌척수액 검사 결과, Gram stain 결과, 그리고 초콜릿 한천 배지에서만 자라는 특이한 성장 양상 모두 *Haemophilus influenzae*에 의한 감염과 일치합니다. 이 세균은 어린아이들에게서 *bacterial meningitis*를 일으키는 것으로 알려져 있으며, 초콜릿 한천 배지에 포함된 특정 성장 인자를 필요로 하기 때문에 혈액 한천 배지에서는 자라지 않습니다.'}


In [ ]:
from unsloth import FastLanguageModel

# LoRA 병합/추론 최적화 모델 불러오기
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="nanyaas/deepseek-r1-medicalQA-Qwen_first",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)  # 추론 모드 전환


==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.5.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
def make_prompt(question):
    return f"""
아래는 특정 작업에 대한 지침과 함께, 추가적인 맥락을 제공하는 입력이 주어져 있습니다.
요청에 적절히 응답하는 답변을 작성하세요.
답변을 작성하기 전에 질문을 신중히 분석하고, 단계적인 사고 과정을 통해 논리적이고 정확한 결론에 도달하세요.

### 지침:
당신은 임상 추론, 진단, 치료 계획에 대해 고도의 전문 지식을 갖춘 의료 전문가입니다.
다음 의학 질문에 대해 **한국어로만** 답변해 주세요.

### 질문:
{question}

### 응답:
<think>
"""


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install bert-score sentence-transformers


데이터셋 100개

In [ ]:

from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import pandas as pd

# ✅ 1. 문장 임베딩 모델 로드 (코사인 유사도용)
sbert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# ✅ 2. 평가용 데이터 준비
predictions = []
references = []
questions = []

for item in tqdm(dataset):
    q = item["Question"]
    gt = item["Response"]

    # 프롬프트 생성
    prompt = make_prompt(q)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=600)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_full = response.split("### 응답:")[-1].strip()
    pred = pred_full.split("</think>")[-1].strip() if "</think>" in pred_full else pred_full

    # 결과 저장
    questions.append(q)
    predictions.append(pred)
    references.append(gt)

# ✅ 3. BERTScore 계산 (의미 기반 정밀 유사도)
P, R, F1 = bert_score(predictions, references, lang="ko", verbose=True)
bert_f1_scores = [round(f.item(), 4) for f in F1]
avg_bert_f1 = F1.mean().item() * 100

# ✅ 4. 코사인 유사도 계산 (문장 임베딩 유사도)
cosine_scores = []
for pred, ref in zip(predictions, references):
    emb1 = sbert_model.encode(pred, convert_to_tensor=True)
    emb2 = sbert_model.encode(ref, convert_to_tensor=True)
    sim = util.pytorch_cos_sim(emb1, emb2).item()
    cosine_scores.append(round(sim, 4))
avg_cosine = sum(cosine_scores) / len(cosine_scores) * 100

# ✅ 5. 결과 정리
df = pd.DataFrame({
    "Question": questions,
    "Prediction": predictions,
    "Reference": references,
    "BERTScore_F1": bert_f1_scores,
    "CosineSimilarity": cosine_scores
})

# ✅ 6. 요약 출력
print(f"✅ 평균 의미 유사도 (BERTScore F1): {avg_bert_f1:.2f}%")
print(f"✅ 평균 코사인 유사도: {avg_cosine:.2f}%")

# ✅ 7. (선택) CSV로 저장
df.to_csv("evaluation_results.csv", index=False)


100%|██████████| 99/99 [1:01:33<00:00, 37.31s/it]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 1.68 seconds, 59.01 sentences/sec
✅ 평균 의미 유사도 (BERTScore F1): 70.91%
✅ 평균 코사인 유사도: 73.18%


데이터셋 200개

In [ ]:

from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import pandas as pd

# ✅ 1. 문장 임베딩 모델 로드 (코사인 유사도용)
sbert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# ✅ 2. 평가용 데이터 준비
predictions = []
references = []
questions = []

for item in tqdm(dataset):
    q = item["Question"]
    gt = item["Response"]

    # 프롬프트 생성
    prompt = make_prompt(q)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=600)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred_full = response.split("### 응답:")[-1].strip()
    pred = pred_full.split("</think>")[-1].strip() if "</think>" in pred_full else pred_full

    # 결과 저장
    questions.append(q)
    predictions.append(pred)
    references.append(gt)

# ✅ 3. BERTScore 계산 (의미 기반 정밀 유사도)
P, R, F1 = bert_score(predictions, references, lang="ko", verbose=True)
bert_f1_scores = [round(f.item(), 4) for f in F1]
avg_bert_f1 = F1.mean().item() * 100

# ✅ 4. 코사인 유사도 계산 (문장 임베딩 유사도)
cosine_scores = []
for pred, ref in zip(predictions, references):
    emb1 = sbert_model.encode(pred, convert_to_tensor=True)
    emb2 = sbert_model.encode(ref, convert_to_tensor=True)
    sim = util.pytorch_cos_sim(emb1, emb2).item()
    cosine_scores.append(round(sim, 4))
avg_cosine = sum(cosine_scores) / len(cosine_scores) * 100

# ✅ 5. 결과 정리
df = pd.DataFrame({
    "Question": questions,
    "Prediction": predictions,
    "Reference": references,
    "BERTScore_F1": bert_f1_scores,
    "CosineSimilarity": cosine_scores
})

# ✅ 6. 요약 출력
print(f"✅ 평균 의미 유사도 (BERTScore F1): {avg_bert_f1:.2f}%")
print(f"✅ 평균 코사인 유사도: {avg_cosine:.2f}%")

# ✅ 7. (선택) CSV로 저장
df.to_csv("evaluation_results_200.csv", index=False)


100%|██████████| 198/198 [2:03:50<00:00, 37.53s/it]


calculating scores...
computing bert embedding.


  0%|          | 0/7 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/4 [00:00<?, ?it/s]

done in 2.83 seconds, 70.01 sentences/sec


✅ 평균 의미 유사도 (BERTScore F1): 70.96%
✅ 평균 코사인 유사도: 72.56%
